# Uma vez os imóveis em posse da empresa, qual o melhor momento para vendê-los e qual seria o preço da venda?
Construir uma tabela com recomendações de venda com acrescimo de 10 ou 30%.

- Agrupar os imóveis por região (zipcode) e por sazonalidade (season).
- Dentro de cada região e sazonalidade, calcular a mediana de preço e encontrar a melhor season para vender o imóvel.
- Condições de venda:
    - Se o preço da compra for maior que a mediana da região+sazonalidade. O preço da venda será igual ao peço da compra + 10%
    - Se o preço da compra for menor que a mediana da região+sazonalidade. O preço da venda será igual ao peço da compra + 30%

In [4]:
#Libraries
import pandas as pd

#Functions
def get_data(path):
    data = pd.read_csv(path)
    
    return data

#Loading data
data = get_data('datasets/kc_house_data.csv')
purchase_recommended = get_data('datasets/purchase_table.csv')

#transformation
data['date'] = pd.to_datetime(data['date'])

pd.options.display.float_format = '{:.2f}'.format

In [5]:
df1 = data[['id', 'zipcode', 'date', 'price']].copy()

df1

,id,zipcode,date,price
0,7129300520,98178,2014-10-13,221900.00
1,6414100192,98125,2014-12-09,538000.00
2,5631500400,98028,2015-02-25,180000.00
3,2487200875,98136,2014-12-09,604000.00
4,1954400510,98074,2015-02-18,510000.00
...,...,...,...,...
21608,263000018,98103,2014-05-21,360000.00
21609,6600060120,98146,2015-02-23,400000.00
21610,1523300141,98144,2014-06-23,402101.00
21611,291310100,98027,2015-01-16,400000.00


In [6]:
# Definindo nova feature 'season' para calculo das medianas.
df1['month']= df1['date'].dt.month

df1['season'] = 'NA'

for i in range( len(df1) ):
    #Outono
    if (df1.loc[i, 'month'] == 9) | (df1.loc[i, 'month']==10) | (df1.loc[i, 'month']==11):
        df1.loc[i,'season'] = 'Autumn'
    #inverno
    elif (df1.loc[i, 'month'] == 12) | (df1.loc[i, 'month']==1) | (df1.loc[i, 'month']==2):
        df1.loc[i,'season'] = 'Winter'
    #primavera
    elif (df1.loc[i, 'month'] == 3) | (df1.loc[i, 'month']==4) | (df1.loc[i, 'month']==5):
        df1.loc[i,'season'] = 'Spring'
    #verão
    elif (df1.loc[i, 'month'] == 6) | (df1.loc[i, 'month']==7) | (df1.loc[i, 'month']==8):
        df1.loc[i,'season'] = 'Summer'
df1

,id,zipcode,date,price,month,season
0,7129300520,98178,2014-10-13,221900.00,10,Autumn
1,6414100192,98125,2014-12-09,538000.00,12,Winter
2,5631500400,98028,2015-02-25,180000.00,2,Winter
3,2487200875,98136,2014-12-09,604000.00,12,Winter
4,1954400510,98074,2015-02-18,510000.00,2,Winter
...,...,...,...,...,...,...
21608,263000018,98103,2014-05-21,360000.00,5,Spring
21609,6600060120,98146,2015-02-23,400000.00,2,Winter
21610,1523300141,98144,2014-06-23,402101.00,6,Summer
21611,291310100,98027,2015-01-16,400000.00,1,Winter


In [7]:
# Aplicando calculo das medianas por 'zipcode' e 'season'
medianprice_zipseason = df1[['price','zipcode','season']].groupby(['zipcode','season']).median().reset_index()
medianprice_zipseason.columns=['zipcode', 'season', 'median_price']
medianprice_zipseason = medianprice_zipseason.pivot(index=['zipcode'], columns=['season'], values='median_price').reset_index()
medianprice_zipseason.columns=['zipcode', 'Autumn', 'Spring', 'Summer', 'Winter']
medianprice_zipseason

#Qual o 'season' com a maior mediana?
medianprice_zipseason['best_season'] = 'NA'
for i in range( len(medianprice_zipseason) ):
    medianprice_zipseason.loc[i, 'best_season'] = medianprice_zipseason.loc[i,['Autumn','Spring','Summer','Winter']].sort_values(ascending=False).index.to_list()[0]

medianprice_zipseason

,zipcode,Autumn,Spring,Summer,Winter,best_season
0,98001,249900.00,262000.00,262500.00,260000.00,Summer
1,98002,235000.00,235000.00,230000.00,238000.00,Winter
2,98003,250000.00,277500.00,271000.00,266000.00,Spring
3,98004,980000.00,1210000.00,1149000.00,1195500.00,Spring
4,98005,813000.00,776225.00,740000.00,747500.00,Autumn
...,...,...,...,...,...,...
65,98177,574750.00,555250.00,515000.00,557500.00,Autumn
66,98178,290500.00,286250.00,265475.00,273500.00,Autumn
67,98188,270000.00,267500.00,259500.00,261000.00,Autumn
68,98198,266750.00,265000.00,262000.00,267475.00,Winter


In [8]:
# Para obter o best_season de cada imóvel, vamos fazer um merge entre df1 (que possui todos os imóveis) e medianprice_zipseason (que possui a melhor season para venda de cada zipcode)
df2 = pd.merge(df1, medianprice_zipseason, on='zipcode', how='inner')
df2

,id,zipcode,date,price,month,season,Autumn,Spring,Summer,Winter,best_season
0,7129300520,98178,2014-10-13,221900.00,10,Autumn,290500.00,286250.00,265475.00,273500.00,Autumn
1,4060000240,98178,2014-06-23,205425.00,6,Summer,290500.00,286250.00,265475.00,273500.00,Autumn
2,4058801670,98178,2014-07-17,445000.00,7,Summer,290500.00,286250.00,265475.00,273500.00,Autumn
3,2976800796,98178,2014-09-25,236000.00,9,Autumn,290500.00,286250.00,265475.00,273500.00,Autumn
4,6874200960,98178,2015-02-27,170000.00,2,Winter,290500.00,286250.00,265475.00,273500.00,Autumn
...,...,...,...,...,...,...,...,...,...,...,...
21608,2525049086,98039,2014-10-03,2720000.00,10,Autumn,2450000.00,1800000.00,1950000.00,1865000.00,Autumn
21609,2525049113,98039,2014-07-25,1950000.00,7,Summer,2450000.00,1800000.00,1950000.00,1865000.00,Autumn
21610,3262300485,98039,2015-04-21,2250000.00,4,Spring,2450000.00,1800000.00,1950000.00,1865000.00,Autumn
21611,6447300365,98039,2014-11-13,2900000.00,11,Autumn,2450000.00,1800000.00,1950000.00,1865000.00,Autumn


In [9]:
# Aplicando cálculo conforme condição de venda para a melhor 'season' da regiao do imóvel.
df2['best_median_price'] = 0
df2['sale_price']   = 0
df2['profit']       = 0

for i in range( len(df2) ):
    #obter a mediana do best_season
    df2.loc[i,'best_median_price'] = df2.loc[i, df2.loc[i, 'best_season']]
    #calculo preço de venda
    if df2.loc[i,'price'] > df2.loc[i, 'best_median_price']:
        df2.loc[i,'sale_price'] = df2.loc[i,'price']+df2.loc[i,'price']*0.1
    else:
        df2.loc[i,'sale_price'] = df2.loc[i,'price']+df2.loc[i,'price']*0.3
#calculo de profit    
df2['profit'] = df2['sale_price']-df2['price']

In [12]:
#Montagem da tabela de sales recommendation
sale_recommendation = df2[['id','zipcode','best_season','best_median_price','price','sale_price','profit']].copy()
sale_recommendation.columns=['id', 'Zipcode', 'Best Season', 'Best Median Price', 'Purchase Price', 'Sale Price', 'Profit']
sale_recommendation.drop_duplicates(subset='id', keep='first', inplace=True)
sale_recommendation = pd.merge(sale_recommendation, purchase_recommended, on='id', how='inner')
sale_recommendation = sale_recommendation[['id', 'Zipcode', 'Best Season', 'Best Median Price', 'Purchase Price', 'Sale Price', 'Profit']]

sale_recommendation_top10 = sale_recommendation.sort_values('Profit', ascending=False).head(10)
sale_recommendation_top10.to_csv('datasets/sales_table.csv', index=False)
sale_recommendation_top10
#sale_recommendation

,id,Zipcode,Best Season,Best Median Price,Purchase Price,Sale Price,Profit
10498,2470200020,98039,Autumn,2450000.00,1880000.00,2444000.00,564000.00
10481,6447300225,98039,Autumn,2450000.00,1880000.00,2444000.00,564000.00
10501,3262300818,98039,Autumn,2450000.00,1865000.00,2424500.00,559500.00
10490,5425700205,98039,Autumn,2450000.00,1800000.00,2340000.00,540000.00
10492,2525049266,98039,Autumn,2450000.00,1762000.00,2290600.00,528600.00
10487,3738000070,98039,Autumn,2450000.00,1712750.00,2226575.00,513825.00
10489,3262300322,98039,Autumn,2450000.00,1651000.00,2146300.00,495300.00
10495,3025300250,98039,Autumn,2450000.00,1620000.00,2106000.00,486000.00
10482,3262300235,98039,Autumn,2450000.00,1555000.00,2021500.00,466500.00
10491,2525049246,98039,Autumn,2450000.00,1550000.00,2015000.00,465000.00


In [13]:
investment = sale_recommendation_top10['Purchase Price'].sum()
profit = sale_recommendation_top10['Profit'].sum()

print('investment {} / profit {}'.format(investment, profit))

investment 17275750.0 / profit 5182725.0
